In [1]:
import os
import fnmatch
import re

def search_string_in_files(directory, search_string):
    matches = []
    pattern = re.compile(search_string)

    for root, dirnames, filenames in os.walk(directory):
        for filename in fnmatch.filter(filenames, '*'):
            filepath = os.path.join(root, filename)
            try:
                with open(filepath, 'r', encoding='utf-8', errors='ignore') as file:
                    for line_number, line in enumerate(file, 1):
                        if pattern.search(line):
                            matches.append({
                                'file': filepath,
                                'line_number': line_number,
                                'line': line.strip()
                            })
            except Exception as e:
                print(f"Could not read file {filepath}: {e}")

    return matches

# Example usage
directory_to_search = 'C:/Users/alm/git/Imswitch_descspim/'
#directory_to_search = 'C:/Users/alm/AppData/Roaming/Python/Python310/site-packages/thorlabs_apt_device/'
print(directory_to_search)

string_to_search = 'baudrate'

results = search_string_in_files(directory_to_search, string_to_search)

for match in results:
    print(f"Found in {match['file']} on line {match['line_number']}: {match['line']}")


C:/Users/alm/git/Imswitch_descspim/
Found in C:/Users/alm/git/Imswitch_descspim/imswitch\imcontrol\model\SetupInfo.py on line 107: baudrate: int
Found in C:/Users/alm/git/Imswitch_descspim/imswitch\imcontrol\model\interfaces\ESP32Client.py on line 105: def __init__(self, host=None, port=31950, serialport=None, baudrate=115200):
Found in C:/Users/alm/git/Imswitch_descspim/imswitch\imcontrol\model\interfaces\ESP32Client.py on line 134: self.serialdevice = serial.Serial(port=self.serialport, baudrate=baudrate, timeout=1)
Found in C:/Users/alm/git/Imswitch_descspim/imswitch\imcontrol\model\interfaces\ESP32Client.py on line 148: self.serialdevice = serial.Serial(port=iport.device, baudrate=baudrate, timeout=1)
Found in C:/Users/alm/git/Imswitch_descspim/imswitch\imcontrol\model\interfaces\RS232Driver.py on line 22: 'baud_rate': settings["baudrate"],
Found in C:/Users/alm/git/Imswitch_descspim/imswitch\imcontrol\model\managers\ArduinoManager.py on line 14: self.__baudrate = self.__ArduinoInf

In [1]:
import pycobolt
import time

laser = pycobolt.Cobolt06DPL(port='COM12') # Creates a new Cobolt Laser object.
time.sleep(2)
laser.is_connected()

True

In [2]:
dir(laser)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_classify_',
 '_identify_',
 '_timeDiff_',
 'address',
 'analog_modulation',
 'baudrate',
 'clear_fault',
 'connect',
 'constant_current',
 'constant_power',
 'digital_modulation',
 'disconnect',
 'get_current',
 'get_current_setpoint',
 'get_fault',
 'get_mode',
 'get_modualtion_tec_setpoint',
 'get_modulation_current',
 'get_modulation_state',
 'get_modulation_tec',
 'get_ophours',
 'get_power',
 'get_power_setpoint',
 'get_state',
 'interlock',
 'is_connected',
 'is_on',
 'modelnumber',
 'modulation_mode',
 'port',
 'send_cmd',
 'serialnumber',
 'set_current',
 'set_modulation_current_high',

In [ ]:
def setEnabled(self, enabled):      # toggle laser on or off
        if enabled:                             # laser is toggled on 
            self._laser.digital_modulation(0)   # turn off dig. modulation
            self._laser.constant_power()    # set laser to constant power mode
        else:                                   # If laser should be disabled, turn off by setting scanmode to active -> modulation mode
            self._laser.modulation_mode()   # switch to modulation mode
            self._laser.digital_modulation(0)   # but make sure to have digital modulation off

In [23]:
laser.get_power()

666.0

In [24]:
laser.get_modulation_state()

['0', '1']

In [18]:
laser.digital_modulation(enable=1)

'OK'

In [2]:
laser2 = pycobolt.Cobolt06DPL(port='COM12') # Creates a new Cobolt Laser object.
time.sleep(2)
laser2.is_connected()

True

In [5]:
def calculate(req_zstep, posConvFac, getPositions):        
        posConvFac = (1 / posConvFac) * 1000           # to µm/du
        print("posConvFac", posConvFac)        # in µm
        req_zstep_du = req_zstep * posConvFac * 1000   # in du
        print("req_zstep", req_zstep)
        print("req_zstep_du", req_zstep_du)

        pos1s, pos1c, pos2s, pos2c  = [float(i.split()[1]) for i in getPositions]
        print("pos1s", pos1s)
        print("pos1c", pos1c)
        print("pos2s", pos2s)
        print("pos2c", pos2c)
        factor = abs(pos1s - pos2s) / abs(pos1c - pos2c)
        print("factor", factor)
        du_camera = [i for i in range(1, 40)]                          # device units camera
        print("du_camera", du_camera)
        dist_camera = [i*posConvFac for i in du_camera]               # distance camera in µm
        print("dist_camera", dist_camera)
        dist_sample = [i*factor for i in dist_camera]                   # distance sample in µm

        print("dist_sample", dist_sample)
        du_sample = [i/posConvFac for i in dist_sample]                 # device units sample
        print("du_sample", du_sample)
        result = [(i, du) for i, du in enumerate(du_sample) if abs(round(du) - du) <= 0.1]
        print("result", result)
        closest_du = min(result, key=lambda x: abs(x[1] - req_zstep_du))
        print("closest_du", closest_du) 
        sample_zstep = closest_du[1] * posConvFac
        print("sample_zstep", sample_zstep)
        camera_zstep = dist_camera[closest_du[0]]
        print("camera_zstep", camera_zstep)

calculate(1, 34554.96, ("Sample: 21165.9339 µm", "Camera: 6207.9076 µm", "Sample: 21665.92 µm", "Camera: 6386.7242 µm"))

posConvFac 0.028939405515156145
req_zstep 1
req_zstep_du 28.939405515156146
pos1s 21165.9339
pos1c 6207.9076
pos2s 21665.92
pos2c 6386.7242
factor 2.796083249541698
du_camera [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
dist_camera [0.028939405515156145, 0.05787881103031229, 0.08681821654546844, 0.11575762206062458, 0.14469702757578073, 0.17363643309093688, 0.20257583860609302, 0.23151524412124916, 0.2604546496364053, 0.28939405515156147, 0.3183334606667176, 0.34727286618187375, 0.37621227169702987, 0.40515167721218603, 0.43409108272734215, 0.4630304882424983, 0.4919698937576545, 0.5209092992728106, 0.5498487047879668, 0.5787881103031229, 0.607727515818279, 0.6366669213334352, 0.6656063268485913, 0.6945457323637475, 0.7234851378789037, 0.7524245433940597, 0.7813639489092159, 0.8103033544243721, 0.8392427599395282, 0.8681821654546843, 0.8971215709698405, 0.9260609764849966, 0.955000382